In [1]:
# out of the box python modules
import sqlite3
import os
import pathlib

# some
from src.models.make_prediction import get_prediction
from src.data import text_handlers, make_dataset
from src.data import CRUD

# Data wrangling stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

rc('mathtext', default='regular')

%load_ext dotenv
%dotenv

DATABASE_PATH = os.environ['DATABASE_PATH']
MODEL_STORE_PATH = os.environ['MODEL_STORE']
REPORTS_STORE_PATH = os.environ['REPORTS_STORE_PATH']


Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 14


In [2]:
ticker_name = 'FIBRAMQ12.MX'
start_date, end_date = '2022-01-01', '2024-07-20'
with sqlite3.connect(DATABASE_PATH) as conn:
    cur = conn.cursor()
    
    market_data_query = "SELECT * FROM market_data order by ticker, date;".format(start_date, end_date)
    market_data_res = cur.execute(market_data_query)
    market_data_entries = market_data_res.fetchall()
    
    future_dividends_query = "SELECT * FROM dividends;"
    future_dividends_res = cur.execute(future_dividends_query)
    future_dividends_entries = future_dividends_res.fetchall()

full_market_df = pd.DataFrame(market_data_entries, columns=['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'stock_splits'])
full_market_df['date'] = pd.to_datetime(full_market_df['date'])
full_market_df['avg_price'] = full_market_df[['open', 'high', 'low', 'close']].mean(axis=1)
ticker_df = full_market_df.query("ticker == @ticker_name and date >= @start_date & date <= @end_date")
sample = ticker_df.filter(['date', 'ticker', 'avg_price', 'dividends'])

future_dividends_df = pd.DataFrame(future_dividends_entries, columns=['ticker', 'announcement_date', 'dividend_date', 'dividend_amount'])
future_dividends_df['announcement_date'] = pd.to_datetime(future_dividends_df['announcement_date'])
future_dividends_df['dividend_date'] = pd.to_datetime(future_dividends_df['dividend_date'])
ticker_dividends_df = future_dividends_df.query("ticker == @ticker_name")

In [5]:
prediction_df = get_prediction(sample)

c:\Users\alanc\miniconda3\envs\fibra_analyzer_env\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | res_blocks    | ModuleList       | 1.4 K 
---------------------------------------------------
1.4 K     Trainable params
0         Non-trainable params
1.4 K     Total params
0.006     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
c:\Users\alanc\miniconda3\envs\fibra_analyzer_env\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
sample

,date,ticker,avg_price,dividends
7936,2022-01-03,FIBRAMQ12.MX,21.227363,0.0
7937,2022-01-04,FIBRAMQ12.MX,21.293700,0.0
7938,2022-01-05,FIBRAMQ12.MX,20.974087,0.0
7939,2022-01-06,FIBRAMQ12.MX,20.837391,0.0
7940,2022-01-07,FIBRAMQ12.MX,21.223347,0.0
...,...,...,...,...
8558,2024-06-24,FIBRAMQ12.MX,31.217500,0.0
8559,2024-06-25,FIBRAMQ12.MX,30.957500,0.0
8560,2024-06-26,FIBRAMQ12.MX,30.817501,0.0
8561,2024-06-27,FIBRAMQ12.MX,30.945000,0.0


In [7]:
ticker_dividends_df

,ticker,announcement_date,dividend_date,dividend_amount
11,FIBRAMQ12.MX,2024-04-25,2024-06-17,0.525


In [8]:
prediction_df

component,date,avg_price,source
0,2024-02-14,34.137319,actual
1,2024-02-15,34.323373,actual
2,2024-02-16,34.260549,actual
3,2024-02-19,34.231554,actual
4,2024-02-20,34.441773,actual
...,...,...,...
113,2024-07-22,31.432467,prediction
114,2024-07-23,31.210890,prediction
115,2024-07-24,31.086596,prediction
116,2024-07-25,31.241975,prediction


In [ ]:
consolidated = prediction_df.merge(sample, how='left')
consolidated['ticker'] = consolidated['ticker'].ffill()
consolidated['dividends'] = consolidated['dividends'].fillna(0)

In [ ]:
consolidated